In [2]:
# Package
from bs4 import BeautifulSoup
from selenium import webdriver 
import pandas as pd
import time
from selenium.webdriver.common.by import By

In [3]:
# List Penyimpanan
daftarKos = []
daftarAlamat = []
daftarHarga = []
daftarType = []
daftarFasilitas = []

# buat instance webdriver
driver = webdriver.Chrome()


url = 'https://mamikos.com/cari/jakarta-pusat-kota-jakarta-pusat-daerah-khusus-ibukota-jakarta-indonesia/all/bulanan/0-15000000/99?keyword=jakarta%20pusat&suggestion_type=search&rent=2&sort=price,-&price=10000-20000000&singgahsini=1'
driver.get(url)
last_h = 0
    # Scroll untuk load 
for _ in range(50):

        # Scroll awah
        driver.execute_script("window.scrollBy(0, 1000)")
        time.sleep(1)

          # coba klik tombol 'Lihat lebih banyak' kalau muncul
        btns = driver.find_elements(By.XPATH, "//button[contains(., 'Lihat lebih banyak lagi')]")
        if btns:
            driver.execute_script("arguments[0].click();", btns[0])  # klik via JS lebih andal
        time.sleep(1)

    # berhenti jika tinggi halaman tidak bertambah (artinya sudah mentok)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h and not btns:
            break
        last_h = new_h

    # inisialisasi html
html = driver.page_source

    # inisialisasi BeutifulSoup
soup = BeautifulSoup(html, "html.parser")

    # inisiasi bagian discraping
kotak= soup.find_all('div',class_='nominatim-rooms-grid bg-c-grid__item bg-is-col-2')

for banyakKotak in kotak:

        # bagian Kosan
        namaKosan = banyakKotak.find('div' , class_="kost-rc__info")
        kosan = namaKosan.find('span', class_="rc-info__name bg-c-text bg-c-text--body-4").get_text()
        daftarKos.append(kosan)

        # bagian Lokasi
        lokasiKosan = banyakKotak.find('div' , class_="kost-rc__info")
        lokasi = lokasiKosan.find('span', class_="rc-info__location bg-c-text bg-c-text--body-3").get_text()
        daftarAlamat.append(lokasi)   

        # Type Koasn
        typeKosan = banyakKotak.find('div' , class_="rc-overview__label bg-c-label bg-c-label--rainbow bg-c-label--rainbow-white").get_text()
        daftarType.append(typeKosan)

        facility_divs = banyakKotak.find_all("div", class_="rc-facilities")
        # Loop setiap div dan ambil semua span di dalamnya
        for div in facility_divs:
            spans = div.find_all("span", {"data-testid": "roomCardFacilities-facility"})
            fasilitas = [span.get_text(strip=True) for span in spans]
            daftarFasilitas.append(fasilitas)
        
        # Harga Koasan
        hargaRumah = banyakKotak.find('div' , class_="rc-price__real")
        harga = hargaRumah.find('span').get_text()
        daftarHarga.append(harga)

driver.close()



In [4]:
# Hitung Hasil Scrapping
print(len(daftarHarga))
print(len(daftarType))
print(len(daftarKos))
print(len(daftarAlamat))
print(len(daftarFasilitas))


236
236
236
236
236


In [5]:
# Saving Csv
data = {
    'nama_kosan':daftarKos,
    'lokasi':daftarAlamat,
    'type_kosan':daftarType,
    'fasilitas':daftarFasilitas,
    'harga':daftarHarga
}

df = pd.DataFrame(data)
df.to_csv('dataKosanJaKartaPusat.csv',index= False)